## Patch Agent Demo

This notebook demonstrates applying mediator recommendations with the `PatchAgent`.

In [1]:
from importlib import import_module
from pathlib import Path
import json
import sqlite3

from app.factories.agent import AgentFactory
from app.factories.logging_provider import LoggingProvider, ConversationLog
from app.utilities.schema import initialize_database
from app.utilities.snapshots.snapshot_writer import SnapshotWriter
from app.enums.agent_enums import AgentRole
from app.utilities import db

# Explicitly create a shared in-memory SQLite connection
shared_conn = sqlite3.connect(":memory:", check_same_thread=False)

# Assign your shared connection explicitly to the db module
db._CONN = shared_conn
db.DB_PATH = ":memory:"

# Initialize the schema on the shared connection
initialize_database(reset=True)

# Import necessary extensions
import_module("app.extensions.agents")
import_module("app.extensions.tool_providers")

demo = Path("demo_patch.py")
demo.write_text("value = 1\n")

# Now explicitly pass the SAME shared SQLite connection to the LoggingProvider
logger = LoggingProvider(db_path=":memory:", connection=shared_conn)
writer = SnapshotWriter(root="demo_snap")

# Instantiate patch agent
agent = AgentFactory.create(
    "patch", target=str(demo), logger=logger, snapshot_writer=writer
)

# Create mediator recommendation
conv = ConversationLog(
    experiment_id="exp",
    round=0,
    agent_role=AgentRole.MEDIATOR,
    target=str(demo),
    content=json.dumps([{"op": "replace", "from": "1", "to": "2"}]),
    originating_agent="mediator",
    intervention=True,
)

# Run the agent with recommendations
agent.run(recommendations=[conv])

# Retrieve metrics
metrics = agent.quality_logs[0]

In [ ]:
print(f"Cyclomatic Complexity: {metrics.cyclomatic_complexity}")
print(f"Maintainability Index: {metrics.maintainability_index}")

Cyclomatic Complexity: 1.0
Maintainability Index: 90.0
